In [22]:
# Import the data
qiime_taxonomy = '/Users/claranordquist/Documents/Universitetet/HT24/Tillämpad_bioinformatik/Applied-bioinformatics/05_Plot_the_results/01_Data/Silva/Illumina_simulated/illumina_silva_clean.csv'
output_taxonomy = '/Users/claranordquist/Documents/Universitetet/HT24/Tillämpad_bioinformatik/Applied-bioinformatics/05_Plot_the_results/01_Data/Silva/Illumina_simulated/illumina_silva_names_fixed.csv'

# Import packages
import pandas as pd
import numpy as np
import regex as re

## If the taxonomy has been made for multiple sequences at a time

In [24]:
# Read the input data
qiime_read = pd.read_csv(qiime_taxonomy, sep=',')
taxonomic_levels = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']

qiime_read.set_index('index', inplace=True)
qiime_read[taxonomic_levels] = qiime_read['Taxonomy'].str.split(';', expand=True)
# qiime_read[['Genus1', 'Species']] = qiime_read['Species1'].str.split(' ', expand=True)
# qiime_read.drop(['Taxonomy', 'Genus1', 'Species1'], axis=1, inplace=True)
qiime_read.drop(['Taxonomy'], axis=1, inplace=True)
qiime_read['Kingdom'].replace(np.NaN, 'Unassigned', inplace=True)

## If the taxonomy has been made for one sequence at a time

In [2]:
# Read the input data
# qiime_read = pd.read_csv(qiime_taxonomy, sep=' ', skiprows=1, header=None)
# qiime_read.columns = ['Feature ID', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Genus1', 'Species', 'Confidence']
# qiime_read.set_index('Feature ID', inplace=True)

# OR
qiime_read = pd.read_csv(qiime_taxonomy, sep=' ', skiprows=1, header=None)
qiime_read.columns = ['Feature ID', 'Taxonomy', 'Confidence']
taxonomic_levels = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']

qiime_read.set_index('Feature ID', inplace=True)
qiime_read[taxonomic_levels] = qiime_read['Taxonomy'].str.split(';', expand=True)
qiime_read.drop(['Taxonomy'], axis=1, inplace=True)

In [3]:
# Loop through the whole dataframe
# for column in qiime_read.columns[1:-2]:
#     for row in qiime_read.index:
#         value = qiime_read.loc[row, column]
#         # Check if value is a string before searching for '__'
#         if isinstance(value, str) and '__' not in value:
#             qiime_read.loc[row, 'Confidence'] = value
#             qiime_read.loc[row, column] = np.NaN

# for column in qiime_read.columns[:-1]:
#     for row in qiime_read.index:
#         value = qiime_read.loc[row, column]
#         if isinstance(value, str):
#             qiime_read.loc[row, column] = value.replace(';', '').strip()

qiime_read['Confidence'] = qiime_read['Confidence'].apply(lambda x: float(x))

In [4]:
qiime_read.drop('Genus1', axis=1, inplace=True)

## Run this in either case

In [14]:
# Delete the taxonomical levels from the names (k__Bacteria --> Bacteria)
def delete_prefix(name):
    '''Deletes the taxonomical prefix in classifications
    k_Bacteria --> Bacteria'''
    if type(name) == str:
        return re.sub('.{1}__', '', name)

# Take away things following _ (Akkermansia muciniphila_D_776786 --> Akkermansia muciniphila)
def extract_name(name):
    '''Deletes all characters following _
    Akkermansia muciniphila_D_776786 --> Akkermansia muciniphila'''
    if type(name) == str:
        return re.sub(r'_.*', '', name)

In [25]:
a = qiime_read.copy(deep=True)

In [26]:
for column in qiime_read.columns:
    for row in qiime_read.index:
        value = delete_prefix(a.loc[row, column])
        name = extract_name(value)
        a.loc[row, column] = name

In [27]:
a.to_csv(output_taxonomy)